In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from matplotlib import cm

# Import custom functions
from util import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# Define colors
colormap = cm.tab20.colors
viridis = cm.get_cmap('viridis', 12)
process_colors = [viridis(0.95), viridis(0.6), viridis(
    0.45), 'red', 'orange', 'grey', viridis(0.2)]

# Load the data
with open('../../data/monte_carlo_individual.pkl', 'rb') as f:
    monte_carlo_individual = pickle.load(f)

with open('../../data/monte_carlo_all_params.pkl', 'rb') as f:
    monte_carlo_dict = pickle.load(f)

# X = excavation
# T = transportation
# B = beneficiation
# R = reactor
# E = electrolysis
# L = liquefaction
# S = storage

storage_label = 0
liquefaction_label = 1
electrolysis_label = 2
transportation_label = 3
excavation_label = 4
beneficiation_label = 5
hydrogen_reduction_label = 6

ilmenite_wt = np.linspace(1, 16, 31)

# Storage, Liquefaction, Electrolysis, Transportation, Excavation, Beneficiation, Reactor
process_indexes = [storage_label, liquefaction_label, electrolysis_label,
                   transportation_label, excavation_label, beneficiation_label, hydrogen_reduction_label]
process_labels = ['Storage', 'Liquefaction', 'Electrolysis',
                  'Transportation', 'Excavation', 'Beneficiation', 'Hydrogen reduction']

process_parameters = {
    'Excavation': ['cohCoeff', 'intAngle', 'extAngle'],
    'Transportation': ['motor_efficiency', 'mRover'],
    'Beneficiation': ['enrichment_factor', 'benef_ilmenite_recovery'],
    'Hydrogen reduction': ['batch_reaction_time_in_hours', 'CFI_thickness', 'HTMLI_thickness', 'delta_T_insulation', 'reactor_heat_up_time_in_hours', 'T_regolith_in', 'T_pre_heater', 'enrichment_factor', 'benef_ilmenite_recovery'],
    'Electrolysis': ['system_efficiency'],
    'Liquefaction': ['cryocooler_efficiency', 'T_hot_reservoir_carnot_cycle', 'T_of_incoming_oxygen'],
    'Storage': ['vip_thickness', 'vip_thermal_conductivity', 'vip_emissivity', 'cryocooler_efficiency_storage']
}

# Overall uncertainty

In [ ]:
total_df = pd.DataFrame(monte_carlo_dict[1], columns=np.linspace(1, 16, 31))
plt.figure(figsize=(6, 4))
plt.gca().grid(axis='y')
sns.boxplot(data=total_df,color='C0',width=0.75, fliersize=0.5, linewidth=0.9, showfliers=False, dodge=False)
plt.gca().set_xlabel('Ilmenite head grade [wt%]')
plt.gca().set_ylabel('kWh/kg LOX')

plt.xticks([0, 4, 8, 12, 16, 20, 24, 28],[1, 3, 5, 7, 9, 11, 13, 15])
plt.gca().set_xlim((-0.55, 28.55))
plt.gca().set_ylim((0., 325))
plt.gca().set_xlim((-0.55, 28.55))

plt.savefig('systematics_quantiles.png', bbox_inches='tight', dpi=400)

In [ ]:
total_df = pd.DataFrame(monte_carlo_dict[1], columns=np.linspace(1, 16, 31))
plt.figure(figsize=(6, 4))
plt.gca().grid(axis='y')
sns.boxplot(data=total_df,color='C0',width=0.75, fliersize=0.5, linewidth=0.9, showfliers=True, dodge=False)
plt.gca().set_xlabel('Ilmenite head grade [wt]%')
plt.gca().set_ylabel('kWh/kg LOX')

plt.xticks([0, 4, 8, 12, 16, 20, 24, 28],[1, 3, 5, 7, 9, 11, 13, 15])
plt.gca().set_xlim((-0.55, 28.55))
plt.gca().set_ylim((0., 525))
plt.gca().set_xlim((-0.55, 28.55))

plt.savefig('systematics_quantiles_w_fliers.png', bbox_inches='tight', dpi=400)

# Per process

In [ ]:
plt.figure(figsize=(4, 3))
for l in range(0, 7):
    median = np.median(monte_carlo_dict[1][:], axis=0)
    process_median = np.median(monte_carlo_dict[0][:, l, :], axis=0)
    q1 = np.quantile(monte_carlo_dict[0][:, l, :], 0.25, axis=0)
    q3 = np.quantile(monte_carlo_dict[0][:, l, :], 0.75, axis=0)
    
    plt.fill_between(x=ilmenite_wt, y1=q1/median*100, y2=q3/median*100, alpha=0.5, color=process_colors[l], label=process_labels[l])
    plt.errorbar(x=ilmenite_wt, y=process_median/median*100, color=process_colors[l], fmt='-', markersize=2)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.gca().set_xlabel('Ilmenite wt%')
plt.gca().set_ylabel('Relative uncertainty per process [%]')
plt.gca().grid(axis='y')
plt.savefig('systematics_per_process.png', dpi=300, bbox_inches='tight')
plt.show()

# Per parameter

In [ ]:
non_constant_keys = []
for key in monte_carlo_individual.keys():
    q1 = np.quantile(monte_carlo_individual[key][1], 0.25, axis=0)
    q3 = np.quantile(monte_carlo_individual[key][1], 0.75, axis=0)
    median = np.median(monte_carlo_individual[key][1], axis=0)
    
    q1_av = np.mean(q1, axis=0)
    q3_av = np.mean(q3, axis=0)
    median_av = np.mean(median, axis=0)

    q1_std = np.std(q1/median*100, axis=0)
    q3_std = np.std(q3/median*100, axis=0)
    
    q1_av = np.mean(q1/median*100, axis=0)
    q3_av = np.mean(q3/median*100, axis=0)    
    
    q1_diff = (np.max(q1/median*100) - np.min(q1/median*100))/2
    q3_diff = (np.max(q3/median*100) - np.min(q3/median*100))/2
    
    #plt.plot(ilmenite_wt, q1/median*100, label=f'q1 {key}')
    #plt.plot(ilmenite_wt, q3/median*100, label=f'q1 {key}')
    
    extra = ''
    diff = (q1_diff + q3_diff)/2

    if (diff > 1.):
        extra = f' --> not constant, diff {q1_diff:.2f}, {q3_diff:.2f}'
        non_constant_keys.append(key)
    print(key, f'q1: {np.around(q1_av,2)}, q2: {np.around(q3_av,2)}, {extra}')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(
    14, 6), sharex=False, gridspec_kw={'hspace': 0.2, 'wspace': 0.15}, sharey=True)

df = pd.DataFrame(
    {'Process': [], 'Relative uncertainty [%]': [], 'Parameter': []})

process_label = process_labels[6]
process_index = process_indexes[6]

for i, ax in enumerate(axs.ravel()):
    key = non_constant_keys[i]
    label = find_new_label_name(key)
    color = find_label_color(key)
    
    mu = np.mean(monte_carlo_individual[key][1][:], axis=0)
    std = np.std(monte_carlo_individual[key][1][:], axis=0)
    q1 = np.quantile(monte_carlo_individual[key][1][:], 0.25, axis=0)
    q3 = np.quantile(monte_carlo_individual[key][1][:], 0.75, axis=0)
    median = np.median(monte_carlo_individual[key][1][:], axis=0)
    
    print(label, np.around(np.mean(std/mu*100), 2))
    new_row = pd.DataFrame({'Process': [process_label],
                                'Relative uncertainty [%]': [np.around(std / mu * 100, 4)],
                                'Parameter': [label]})
    df = pd.concat([df, new_row], ignore_index=True)
        
    ax.fill_between(x=ilmenite_wt, y1=q1/median*100, y2=q3/median*100, alpha=0.4, color=color, label=label)
    ax.errorbar(ilmenite_wt, y=median/median*100, label='Median [%]', color=color)
    
    ax.grid(axis='y')
    ax.legend()

axs[0, 0].set_ylabel(
    'Q1 and Q3 quantiles \n relative to median [%]')
axs[1, 0].set_ylabel(
    'Q1 and Q3 quantiles \n relative to median [%]')
for _ in range(0, 3):
    axs[1, _].set_xlabel('Ilmenite [wt%]')

plt.savefig('systematics_per_process_per_param.png',
            dpi=400, bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(
    14, 6), sharex=True, gridspec_kw={'hspace': 0.2, 'wspace': 0.15}, sharey=True)

fig.delaxes(axs[1, 3])
df = pd.DataFrame(
    {'Process': [], 'Relative uncertainty [%]': [], 'Parameter': []})

for i, ax in enumerate(axs.ravel()):
    if i == 7:
        break
    process_label = process_labels[i]
    process_index = process_indexes[i]
    print(f'----- {process_label}')
    for key in process_parameters[process_label]:
        label = find_new_label_name(key)
        color = find_label_color(key)
        mu = np.mean(monte_carlo_individual[key][0][:, process_index], axis=0)
        std = np.std(monte_carlo_individual[key][0][:, process_index], axis=0)
        q1 = np.quantile(monte_carlo_individual[key][0][:, process_index], 0.25, axis=0)
        q3 = np.quantile(monte_carlo_individual[key][0][:, process_index], 0.75, axis=0)
        median = np.median(monte_carlo_individual[key][0][:, process_index], axis=0)
        
        print(label, np.around(np.mean(std/mu*100), 2))
        new_row = pd.DataFrame({'Process': [process_label],
                                'Relative uncertainty [%]': [np.around(std / mu * 100, 4)],
                                'Parameter': [label]})
        df = pd.concat([df, new_row], ignore_index=True)
        
        ax.fill_between(x=ilmenite_wt, y1=q1/median*100, y2=q3/median*100, alpha=0.5, color=color, label=label)
        #ax.errorbar(ilmenite_wt, y=std/mu*100, label=label, color=color)

    if process_label == 'Hydrogen reduction':
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    else:
        ax.legend(loc='upper left', bbox_to_anchor=(.03, 0.9), borderaxespad=0.)
    ax.set_title(process_label)
    ax.grid(axis='y')
    #ax.set_ylim((0.5, 1.75))
axs[0, 0].set_ylabel(
    'Relative uncertainty [%] \n ($\sigma_{process}/\mu_{process}$) ')
axs[1, 0].set_ylabel(
    'Relative uncertainty [%] \n($\sigma_{process}/\mu_{process}$) ')
for _ in range(0, 3):
    axs[1, _].set_xlabel('Ilmenite wt%')
axs[0, 3].set_xlabel('Ilmenite wt%')
plt.savefig('systematics_per_process_per_param.png',
            dpi=300, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = plt.gca()
i = 6

process_label = process_labels[i]
process_index = process_indexes[i]

for i, key in enumerate(process_parameters[process_label]):
    label = find_new_label_name(key)
    color = colormap[i]  # find_label_color(key)
    mu = np.mean(monte_carlo_individual[key][0][:, process_index], axis=0)
    std = np.std(monte_carlo_individual[key][0][:, process_index], axis=0)
    ax.errorbar(ilmenite_wt, y=std/mu*100, label=label, color=color)
    if np.std(std/mu) > 0.:
        print(key, np.around(np.std(std/mu*100), 2))

if process_label == 'Hydrogen reduction':
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
else:
    ax.legend(loc='upper left', bbox_to_anchor=(.03, 0.9), borderaxespad=0.)
    ax.set_title(process_label)

ax.set_ylabel('Relative uncertainty [%] \n ($\sigma/\mu$) ')
ax.set_title('Hydrogen reduction')
ax.grid(axis='y')

ax.set_xlabel('Ilmenite wt%')
plt.savefig('systematics_hydrogen_reduction.png', dpi=300, bbox_inches='tight')

# At 10 % ilmenite

In [ ]:
print('Value at 10% ilmenite in kWh/kg LOX')
print('-'*36)
for i, l in enumerate(process_labels):
    print(l, np.around(mu_at_10_per_process, 3)[i])

print(' '*36)
print('Uncertainty at 10% ilmenite in kWh/kg LOX')
print('-'*41)
for i, l in enumerate(process_labels):
    print(l, np.around(stds_at_10_per_process, 3)[i])

print(' '*36)
print('Relative uncertainty at 10% ilmenite')
print('-'*36)
for i, l in enumerate(process_labels):
    print(l, np.around(relative_stds_at_10_per_process, 3)[i])

In [ ]:
stds_per_param = []
relative_stds_per_param = []

for i, l in enumerate(monte_carlo_individual.keys()):
    stds_per_param.append(
        (l, np.around(np.std(monte_carlo_individual[l][1], axis=0)[18], 3)))
    relative_stds_per_param.append(
        (l, np.around(np.std(monte_carlo_individual[l][1], axis=0)[18]/total_mu[18]*100, 3)))

stds_per_param.sort(key=lambda x: x[1], reverse=True)
relative_stds_per_param.sort(key=lambda x: x[1], reverse=True)

In [ ]:
i = np.linspace(1,99,99)
i = i / 100
print(i)
print((i[0:95:3]))

In [ ]:
print('='*36)
print('Relative uncertainty at 10% ilmenite')
print('='*36)

print('5 Largest uncertainty per parameter')
print('-'*36)
for i in range(0, 5):
    print(stds_per_param[i])

print(' '*36)
print('5 Largest relative uncertainty per parameter')
print('-'*44)
for i in range(0, 5):
    print(relative_stds_per_param[i])
